___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
- <Strong> Flavio Maximiliano Herrada Avalos</Strong>
- <Strong> Año </Strong>: 2021
- <Strong> Email: </Strong>  <font color="blue"> `flavio.herrada@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG</p>

### <font color= #2E9AFE> Tema: Modelos basados en Árboles Parte II - Clasificación</font>

### Bosques Aleatorios

"Los árboles tienen un sólo aspecto que previene que sean la herramienta ideal para el aprendizaje predictivo, que es la **inexactitud**" 

Pasos para crear un bosque aleatorio:
- Crear un dataset "bootstrapped"
- Crear un árbol de decisión usando el dataset "bootstrapped", pero sólo usar un subconjunto aleatorio de variables (o columnas) en cada paso. 
- Regresar al paso 1. y repetir 

Gracias al proceso de bootstrapping, el requerimento de dividir los datos en prueba y entrenamiento no es tan estricto. Se recomienda dividir los datos en prueba y entrenamiento cuando se quiere comparar su desempeño contra otros modelos. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.metrics import (accuracy_score,precision_score,recall_score)
import time

In [ ]:
#Importar la base de datos
iris = datasets.load_iris()
X = iris.data
Y = iris.target

plt.scatter(X[:,0],X[:,1], c=Y, s=10, cmap=plt.cm.rainbow,zorder=2)
plt.xlabel('x_p1')
plt.ylabel('x_p2')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0,
                                                    shuffle=True)

In [ ]:
#%% Construccion y entrenamiento de la bolsa de modelos
modelo = RandomForestClassifier(n_estimators=100,
                               criterion='gini',
                               max_depth=10,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               max_features='auto',
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=1)

start_time = time.time()
modelo = modelo.fit(X_train,y_train) # prediccion con la bolsa de modelos
print("--- %s seconds ---" % (time.time() - start_time))
Yhat = modelo.predict(X_test)

In [ ]:
# Evaluacion del modelo
accu = accuracy_score(y_test,Yhat)
prec = precision_score(y_test,Yhat,average='weighted')
reca = recall_score(y_test,Yhat,average='weighted')
print('Accuracy\t Precision\t Recall\n %0.3f\t %0.3f\t %0.3f'%(accu,prec,reca))

In [ ]:
# si yo quisiera ver sólo un árbol
#no se puede imprimir todo el árbol aleatorio
from sklearn import tree
tree.plot_tree(modelo.estimators_[0]) 

In [ ]:
#Buscar un número bueno de árboles a usar
ntrees = np.arange(1,20,1)
R2_s = np.zeros(len(ntrees))
for n in range(len(ntrees)):
    model = RandomForestClassifier(n_estimators=ntrees[n],
                               criterion='gini',
                               max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               max_features='auto',
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=0)
    model = model.fit(X_train,y_train)
    R2_s[n] = model.score(X_train,y_train)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(ntrees,R2_s,c='r',label='ntrees')
plt.ylabel('Accuracy')
plt.xlabel('Trees')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#Usando cross validation y grid search
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(criterion='gini',
                               min_samples_leaf=2,
                               max_features='auto',
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=0)

gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 8),
                                'min_samples_split': range(5, 10, 2),
                                'n_estimators': range(2,5,1)}, 
                  cv=10,
                  scoring='accuracy')

gs.fit(X_train, y_train)


In [ ]:
print(gs.best_params_)

In [ ]:
#crear modelo usando parámetros óptimos
new_model = RandomForestClassifier(n_estimators=2,
                               criterion='gini',
                               max_depth=2,
                               min_samples_split=5,
                               min_samples_leaf=2,
                               max_features='auto',
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=0)
new_model.fit(X_train, y_train)

In [ ]:
# Evaluacion del modelo
yhat = new_model.predict(X_test)
accu = accuracy_score(y_test,yhat)
prec = precision_score(y_test,yhat,average='weighted')
reca = recall_score(y_test,yhat,average='weighted')
print('Accuracy\t Precision\t Recall\n %0.3f\t %0.3f\t %0.3f'%(accu,prec,reca))

**Ventajas**

- Son muy buenos generalizando
- Protejen en contra del sobreajuste (overfitting) gracias a la construcción del bootstrapping 
- También reducen la varianza y por lo tanto mejoran la precisión del modelo
- Funcionan muy bien con variables categóricas y variables continuas
- No se requiere escalamiento previo de variables 
- Manejan muy bien el hecho de que haya datos nulos
- Son modelos robustos ante valores atípicos (outliers)
- Son algoritmos muy estables, cuando hay datos nuevos, el algoritmo no se ve muy afectado. Ya que este nuevo dato puede afectar a un árbol individual, pero es difícil que impacte a todos los árboles. 

**Desventajas**

- Complejidad. Los bosques aleatorios crean muchos árboles y combina sus resultados. Requiere mucho poder computacional y recursos 
- Periodos de entrenamiento largos. Requieren más tiempo de entrenamiento. 